In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



In [24]:

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range=0.2,
                                   shear_range= 0.2,)

valid_datagen = ImageDataGenerator(rescale = 1./255)

train_dir_path = "ocr-data/data/training_data/"
valid_dir_path = "ocr-data/data/testing_data"

train_data = train_datagen.flow_from_directory(train_dir_path,
                                               target_size = (150,150),
                                               class_mode='categorical',
                                               color_mode = 'grayscale',
                                               seed = 42)

valid_data = valid_datagen.flow_from_directory(valid_dir_path,
                                               target_size = (150,150),
                                               class_mode = 'categorical',
                                               color_mode = 'grayscale',
                                               seed = 42)


Found 20628 images belonging to 36 classes.
Found 1008 images belonging to 36 classes.


In [25]:
from pathlib import Path
class_name = np.array(sorted(item.name for item in Path(train_dir_path).glob("*")))
print(class_name)

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [26]:
len(class_name)

36

In [29]:
inputs = layers.Input(shape = (150,150,1), dtype = tf.float64)
x = layers.Conv2D(16,3,activation = "relu")(inputs)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(64,3,activation = "relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(128,3,activation = "relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128,3,activation = "relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(300, activation = "relu")(x)
outputs = layers.Dense(36, activation = "softmax")(x)

model_1 = tf.keras.Model(inputs, outputs)
model_1.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d_32 (Conv2D)          (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 72, 72, 64)        9280      
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 36, 36, 64)        0   

In [30]:
model_1.compile(loss = "categorical_crossentropy",
                optimizer = "adam",
                metrics = ["accuracy"])

history_1 = model_1.fit(train_data,
                      epochs = 5,
                      steps_per_epoch = len(train_data),
                      validation_data = valid_data,
                      validation_steps = len(valid_data))

Epoch 1/5
645/645 [==============================] - 483s 747ms/step - loss: 1.5754 - accuracy: 0.5456 - val_loss: 0.2506 - val_accuracy: 0.9206
Epoch 2/5
645/645 [==============================] - 296s 459ms/step - loss: 0.5910 - accuracy: 0.8219 - val_loss: 0.1542 - val_accuracy: 0.9385
Epoch 3/5
645/645 [==============================] - 270s 418ms/step - loss: 0.4379 - accuracy: 0.8650 - val_loss: 0.1117 - val_accuracy: 0.9524
Epoch 4/5
645/645 [==============================] - 255s 395ms/step - loss: 0.3718 - accuracy: 0.8866 - val_loss: 0.0565 - val_accuracy: 0.9742
Epoch 5/5
645/645 [==============================] - 292s 453ms/step - loss: 0.3249 - accuracy: 0.8959 - val_loss: 0.0665 - val_accuracy: 0.9722


In [35]:
def load_and_prep_data(filename, filesize = [150,150]):
  # read the images
  img = tf.io.read_file(filename)
  # decode the images
  img = tf.io.decode_image(contents=img, channels = 1)
  # resize the image 
  img = tf.image.resize(img , size = filesize)
  img = img/255.
  # expand the dims
  img = tf.expand_dims(img, axis = 0)
  

  return img

     

In [67]:
def evaluate(img):
    f = tf.math.argmax(img, axis = 1)

    return f

In [78]:
exp_path = "b.png"


In [79]:
image = load_and_prep_data(exp_path)
img_class =  model_1.predict(image)
print(img_class)
f_image= evaluate(img_class)


1/1 [==============================] - 0s 21ms/step
[[5.2713892e-11 2.6725164e-22 9.4208917e-17 1.0684853e-14 3.6485923e-15
  7.2607124e-16 1.5338475e-08 6.5007936e-16 9.4074894e-11 1.0604093e-21
  6.5218842e-08 9.9999940e-01 2.9721887e-13 1.1748455e-10 6.9942302e-10
  3.7182920e-11 6.3606096e-14 2.8730721e-10 1.8043740e-14 4.5096757e-22
  7.0642350e-12 5.1298522e-13 4.0102737e-20 6.3087697e-16 3.0314085e-11
  4.9673701e-09 2.7389206e-15 5.2130832e-07 2.4080386e-18 8.0837941e-13
  3.1207008e-14 8.0340562e-16 2.2295146e-15 2.3888629e-17 3.3692127e-15
  1.8651403e-11]]


In [80]:
class_name[f_image]


array(['B'], dtype='<U1')